## Performing a single backpropagation step to updata the parameter values once

**Goal:** In this notebook you will see how to use tensorflow to do a single update step based on stochastic gradient descent with one data point. You will do one forward pass and one backward pass and extract the gradients of intermediate terms in the computational graph. You use them for computing the gradients of the loss w.r.t. the parameters (slope and intercept) which are needed to do one updatestep.

**Usage:** The idea of the notebook is that you try to understand the provided code by running it, checking the output and playing with it by slightly changing the code and rerunning it. 

**Dataset:** You work with a single datapoint of the systolic blood pressure and age data of 33 American women, which is generated in the upper part of the notebook . 

**Content:**

* read book chapter 3.4.1 check how the provided code corresponds to the step by step computations in this chapter. 

* use the tensorflow library to set up the model 
    * define a computational graph containing all intermediate terms and local gradients 
    * do a single forward pass and compute all intermediate terms
    * do a single backward pass and compute all local gradients and use them to compute the gradients of the loss w.r.t. the parameters via chain rule
    * do a single update step of the parameter values
    * verify that the computed values for the gradients and the updated parameter values correspond to the values in chapter 3.4.1.



[open in colab](https://colab.research.google.com/github/tensorchiefs/dl_book/blob/master/chapter_03/nb_ch03_04_tf2_version.ipynb)

#### Install correct TF version (colab only)

In [1]:
# Execute this cell to be sure to have a compatible TF (2.0) version. 
# If you are bold you can skip this cell. 
try: #If running in colab 
  import google.colab
  !pip install tensorflow==2.0.0
except:
  print('Not running in colab')

In [2]:
try: #If running in colab 
    import google.colab
    IN_COLAB = True 
    %tensorflow_version 2.x
except:
    IN_COLAB = False

In [3]:
import tensorflow as tf
if (not tf.__version__.startswith('2')): #Checking if tf 2.0 is installed
    print('Please install tensorflow 2.0 to run this notebook')
print('Tensorflow version: ',tf.__version__, ' running in colab?: ', IN_COLAB)

Tensorflow version:  2.0.0  running in colab?:  True


In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
import tensorflow as tf
print('TF Version:', tf.__version__)

TF Version: 2.0.0


#### Blood Pressure data

Here we read in the systolic blood pressure and the age of the 33 American women in our dataset.



In [5]:
# Blood Pressure data
x = [22, 41, 52, 23, 41, 54, 24, 46, 56, 27, 47, 57, 28, 48, 58,  9, 
     49, 59, 30, 49, 63, 32, 50, 67, 33, 51, 71, 35, 51, 77, 40, 51, 81]
y = [131, 139, 128, 128, 171, 105, 116, 137, 145, 106, 111, 141, 114, 
     115, 153, 123, 133, 157, 117, 128, 155, 122, 183,
     176,  99, 130, 172, 121, 133, 178, 147, 144, 217] 
x = np.asarray(x, np.float32) 
y = np.asarray(y, np.float32)

###  Doing the back propagation by hand for the example

In the next cell we take only one woman of the dataset, because we want to calculate the gradients with only one datapoint. The woman is 58 years old and has a sbp value of 153.

In [6]:
x = x[14]
y = y[14]
print(x)
print(y)

58.0
153.0


Here we define the computational graph with all the intermediate values and gradients in between, because we need them to apply the the chain rule and do the backpropagation. (see figure 3.12 in the book)

In [7]:
# Defining the graph (construction phase)

a_  = tf.Variable(0.0, name='a_var')                       # Variables, with starting values, will be optimized later
b_  = tf.Variable(139.0, name='b_var')                     # we name them so that they look nicer in the graph
x_  = tf.constant(x, name='x_const')                       # Constants, these are fixed tensors holding the data values and cannot be changed by the optimization
y_  = tf.constant(y, name='y_const')  


# We now do it step by step so that we can calculate the intermediate values and gradients
def my_func():
  ax_ = a_* x_
  abx_ = ax_ + b_
  r_ = abx_ - y_
  s_ = tf.square(r_)
  mse_ = tf.reduce_mean(s_)                                 
  return([a_,b_,x_,y_,ax_,abx_,r_,s_,mse_])

In [8]:
a_,b_,x_,y_,ax_,abx_,r_,s_,mse_=my_func()        

#### Simple forward pass

Now, let's do a simple forward pass and print the resulting values for ax, abx, r, s, and the mse.

In [9]:
vals = (ax_,abx_,r_,s_,mse_) # Letting the variables a=0 b=139 flow through the graph
for p in vals:
  print(p)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(139.0, shape=(), dtype=float32)
tf.Tensor(-14.0, shape=(), dtype=float32)
tf.Tensor(196.0, shape=(), dtype=float32)
tf.Tensor(196.0, shape=(), dtype=float32)


#### Extracting the gradients and the updated values

In [10]:

optimizer = tf.keras.optimizers.SGD(0.00002)
with tf.GradientTape(persistent=True) as tape:
  a_,b_,x_,y_,ax_,abx_,r_,s_,mse_=my_func()        
  grad_mse_s = tape.gradient(mse_, s_)
  grad_s_r = tape.gradient(s_, r_)
  grad_r_abx_= tape.gradient(r_, abx_)
  grad_abx_b = tape.gradient(abx_, b_)
  grad_abx_ax = tape.gradient(abx_, ax_)
  grad_ax_a = tape.gradient(ax_, a_)
  gradients = tape.gradient(mse_, [a_,b_])
  optimizer.apply_gradients(zip(gradients,[a_,b_]))  


In [11]:
print(grad_mse_s.numpy(),grad_s_r.numpy(),grad_r_abx_.numpy(),grad_abx_b.numpy(),grad_abx_ax.numpy(),grad_ax_a.numpy())
print(a_.numpy(),b_.numpy())

1.0 -28.0 1.0 1.0 1.0 58.0
0.032479998 139.00056


<img src="https://raw.githubusercontent.com/tensorchiefs/dl_book/master/imgs/ch03_12.pdf.png" width="800" align="left" />  
Compare the results of tensorflow with the results form the book where we did the forward and the backward pass by hand. The forward pass in blue and the backward pass in red.

In [12]:

a_  = tf.Variable(0.0, name='a_var')                       # Variables, with starting values, will be optimized later
b_  = tf.Variable(139.0, name='b_var')                     # we name them so that they look nicer in the graph

with tf.GradientTape(persistent=True) as tape:
  a_,b_,x_,y_,ax_,abx_,r_,s_,mse_=my_func()        
  grads_mse_a_b = tape.gradient(mse_, [a_,b_])
print(grads_mse_a_b)

[<tf.Tensor: id=131, shape=(), dtype=float32, numpy=-1624.0>, <tf.Tensor: id=130, shape=(), dtype=float32, numpy=-28.0>]


#### Compute the gradient of the mse w.r.t to a via the chain rule 

In [13]:
#grad_mse_a 
print(grad_mse_s.numpy()*grad_s_r.numpy()*grad_r_abx_.numpy()*grad_abx_ax.numpy()*grad_ax_a.numpy())

-1624.0


In [14]:
#grad_mse_b 
print(grad_mse_s.numpy()*grad_s_r.numpy()*grad_r_abx_.numpy()*grad_abx_b.numpy())

-28.0


#### Update Formula
Verify that we get the same if we do the upate "by hand". 

In [15]:
a0=0
b0=139
eta=0.00002
print(a0-eta*grads_mse_a_b[0].numpy())
print(b0-eta*grads_mse_a_b[1].numpy())

0.03248
139.00056
